In [1]:
import numpy as np

# Entrada/Salida de datos
data = open('input.txt', 'r').read() # archivo de texto plano de entrada
chars = list(set(data)) # caracteres unicos en el archivo de entrada
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # diccionario: caracter -> indice
ix_to_char = { i:ch for i,ch in enumerate(chars) } # diccionario: indice -> caracter

data has 46 characters, 20 unique.


In [2]:
# Hiperparametros
hidden_size = 100 # tamaño de la capa oculta (cantidad de neuronas)
seq_length = 25 # numero de pasos para desenrollar la RNN (longitud de secuencia)
learning_rate = 1.e-4

In [3]:
# RNN COMUN PELADA COMO LA DEL ERICH
# parametros del modelo (matrices de pesos y sesgos)
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # pesos: entrada -> capa oculta
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # pesos: capa oculta -> capa oculta (memoria)
Why = np.random.randn(vocab_size, hidden_size)*0.01 # pesos: capa oculta -> salida
bh = np.zeros((hidden_size, 1)) # sesgo de la capa oculta
by = np.zeros((vocab_size, 1)) # sesgo de la capa de salida

In [4]:
def lossFun(inputs, targets, hprev):
  """
  Calcula la pérdida (loss) y los gradientes para actualizar los pesos.
  
  Parámetros:
  - inputs: lista de índices de caracteres de entrada
  - targets: lista de índices de caracteres objetivo (siguiente carácter)
  - hprev: estado oculto anterior (memoria de la red)
  
  Retorna:
  - loss: valor de la función de pérdida
  - gradientes para cada parámetro (dWxh, dWhh, dWhy, dbh, dby)
  - último estado oculto
  """
  xs, hs, ys, ps = {}, {}, {}, {} # diccionarios para almacenar valores en cada paso temporal
  hs[-1] = np.copy(hprev) # inicializar con el estado oculto anterior
  loss = 0
  
  # Propagacion hacia adelante (forward pass)
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # codificar entrada en representacion one-hot
    xs[t][inputs[t]] = 1 # poner un 1 en la posicion del caracter actual

    # aca se rompe, con rnn comun, lo que hay que hacer aca es update gate y reset gate
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # calcular estado oculto
      
    ys[t] = np.dot(Why, hs[t]) + by # calcular salida (logits sin normalizar)
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # aplicar softmax: convertir a probabilidades
    loss += -np.log(ps[t][targets[t],0]) # calcular perdida con entropia cruzada
  
  # Propagacion hacia atras (backward pass): calcular gradientes en orden inverso
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0]) # gradiente del siguiente estado oculto
  
  for t in reversed(range(len(inputs))): # recorrer desde el final hasta el inicio
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # gradiente de la funcion softmax con entropia cruzada
    dWhy += np.dot(dy, hs[t].T) # gradiente de Why
    dby += dy # gradiente de by
    dh = np.dot(Why.T, dy) + dhnext # gradiente que llega a la capa oculta
    dhraw = (1 - hs[t] * hs[t]) * dh # gradiente a traves de la funcion tanh
    dbh += dhraw # gradiente de bh
    dWxh += np.dot(dhraw, xs[t].T) # gradiente de Wxh
    dWhh += np.dot(dhraw, hs[t-1].T) # gradiente de Whh
    dhnext = np.dot(Whh.T, dhraw) # gradiente para el paso temporal anterior
  
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # recortar gradientes para evitar explosion
  
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [5]:
def sample(h, seed_ix, n):
  """ 
  Genera una secuencia de caracteres del modelo entrenado.
  
  Parámetros:
  - h: estado oculto (memoria) inicial
  - seed_ix: índice del carácter semilla para comenzar
  - n: cantidad de caracteres a generar
  
  Retorna:
  - lista de índices de caracteres generados
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1 # codificar caracter inicial en one-hot
  ixes = []
  
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh) # actualizar estado oculto
    y = np.dot(Why, h) + by # calcular salida
    p = np.exp(y) / np.sum(np.exp(y)) # convertir a probabilidades con softmax
    ix = np.random.choice(range(vocab_size), p=p.ravel()) # muestrear siguiente caracter
    x = np.zeros((vocab_size, 1))
    x[ix] = 1 # actualizar entrada para el siguiente paso
    ixes.append(ix)
  
  return ixes

In [6]:
# Variables para el entrenamiento
n, p = 0, 0 # n: contador de iteraciones, p: puntero de posición en los datos
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # variables de memoria para Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # perdida inicial estimada

In [7]:
max_iterations = 1000

while n < max_iterations:
  # Si es el primer entrenamiento o si se pasa ya la secuencia se reinicia la capa oculta los pesos
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reiniciar memoria de la RNN
    p = 0 # volver al inicio de los datos

  # inputs son los caracteres pero como indice(numero), targets es uno adelante de inputs, con tamanio de secuencia de length, es lo que queremos predecir
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] # secuencia de entrada
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] # secuencia objetivo (siguiente caracter)

  # Generar texto de muestra cada 100 iteraciones, con la funcion sample, para ver como empieza a aprender patrones
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200) # generar 200 caracteres
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # Calcular perdida y gradientes
  # loss perdida, 
  # gradientes dWxh,dWhh, dWhy, dbh, dby
  # hprev el ultimo estado oculto, que se usa como entrada en la proxima iteracion (memoria)
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    
  smooth_loss = smooth_loss * 0.999 + loss * 0.001 # suavizar la perdida para visualizacion
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # mostrar progreso
  
  # Actualizar parametros usando Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam # acumular cuadrados de gradientes
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # actualizacion Adagrad

  p += seq_length # mover puntero de datos
  n += 1 # incrementar contador de iteraciones

print("entrenamiento RNN comun terminado!")

----
 nfixapcufnroaRTiftptiNnbomtrNfmdfxdmNon xmupunNfTptmxnTmxtbunddvmRoNxaTppTpnnrxRcoNvtapboarcvppxxabuaTdTrxcdftrcupfeopRf NcdReadpoexnbuvTrdcuuT trbNxfotvuiiifurnria udirpox uevmpaefvxo xaimvdemRTRpbaN 
----
iter 0, loss: 74.893305
----
 aTouccNNcaNvpNneNucTbNeacdcNueiTTfNcpptT xvNmbf xcvoTNxxixouieitffrpibNfdp rnaprefdxm cedmoopttrRvtcotacmvNeniboarrxeNmrdbTcdfTdptfrunuxvNxximv vtuemecNTR dafnRtcvaaodmutdTRfiTTNxmmcrao eTNxfaTcfuTcRa 
----
iter 100, loss: 74.885706
----
 RxufmebRtrmnixTofvcTiouRivovnobpxueotNrenmNobrnvceii xncbbtntnt b Nofb tvdTcorrmtnntuubrrppuoRpTTu R uxarxNcueuvmupnRatc uxTimNxcecRmeefbTpxncunrd dnaabeemtiTm TpccactciumidmxtiraxvxrraRruemaibNavndev 
----
iter 200, loss: 74.871125
----
 RivvdimdetnntTveeboxeNicxpTRmNiabb  mTemNtxcaoeTfbxRRoRcodNnTTTRtoprcuTtdpnTbffbneppfcaptcfmeupNnRopcoNpmrvep eRNubrd vrduRdT unifip niiaoxNpnRRxvdnvcuamxuxaRexcvottNximRxtrnbNRvx mdtounTmpcTmvrbRrnxx 
----
iter 300, loss: 74.851429
----
 fofbTtcvntuTar  aomo TxaiaT TfatatpcrRnb

In [8]:
# GRU (con puertas de actualizacion, reseteo y candidato oculto)
# Donde luego tenemos combinacion final de salida

# puerta de actualizacion, es una decision de cuanto del pasado sirve, se mantiene
Wxz = np.random.randn(hidden_size, vocab_size) * 0.01
Whz = np.random.randn(hidden_size, hidden_size) * 0.01
bz = np.zeros((hidden_size, 1))

# puerta de reinicio, es una decision de cuanto del pasado se ignora
Wxr = np.random.randn(hidden_size, vocab_size) * 0.01
Whr = np.random.randn(hidden_size, hidden_size) * 0.01
br = np.zeros((hidden_size, 1))

# Candidato oculto
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))

# Capa de salida, evitando la explosion/desaparicion del gradiente
Why = np.random.randn(vocab_size, hidden_size) * 0.01

In [9]:
# Convierte cualquier valor a rango 0 y 1, usada en las puertas
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [10]:
def lossFun(inputs, targets, hprev):
  """
  Calcula la pérdida y los gradientes para entrenar la GRU.
  
  ¿CÓMO FUNCIONA UNA GRU?
  En cada paso temporal t, la GRU decide inteligentemente:
  
  1. z_t (update gate): "¿Cuánto mantener del pasado vs. cuánto actualizar?"
     z_t = sigmoid(Wxz*x_t + Whz*h_{t-1} + bz)
     Valor cercano a 0 = mantener estado anterior
     Valor cercano a 1 = usar nueva información
  
  2. r_t (reset gate): "¿Cuánto del pasado es relevante para el nuevo candidato?"
     r_t = sigmoid(Wxr*x_t + Whr*h_{t-1} + br)
     Valor cercano a 0 = ignorar estado anterior
     Valor cercano a 1 = usar completamente estado anterior
  
  3. h_tilde (candidato): "¿Cuál sería el nuevo estado si actualizamos?"
     h_tilde = tanh(Wxh*x_t + Whh*(r_t ⊙ h_{t-1}) + bh)
     Nota: r_t filtra selectivamente la información del pasado
  
  4. h_t (nuevo estado): "Interpolación entre pasado y candidato"
     h_t = (1 - z_t) ⊙ h_{t-1} + z_t ⊙ h_tilde
     z_t controla la mezcla: 0=todo pasado, 1=todo nuevo
  
  Parámetros:
  - inputs: lista de índices de caracteres de entrada [seq_length]
  - targets: lista de índices de caracteres objetivo [seq_length] 
  - hprev: estado oculto anterior (memoria inicial) [hidden_size, 1]
  
  Retorna:
  - loss: valor de pérdida (cross-entropy)
  - gradientes: dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby
  - hprev: último estado oculto (para siguiente iteración)
  """
  # Diccionarios para guardar valores en cada paso temporal
  xs, hs, hs_tilde, zs, rs, ys, ps = {}, {}, {}, {}, {}, {}, {}
  hs[-1] = np.copy(hprev)  # Estado inicial
  loss = 0
  
  # ============================================================================
  # FORWARD PASS: Procesar la secuencia de izquierda a derecha
  # ============================================================================
  for t in range(len(inputs)):
    # --- Codificar entrada como vector one-hot ---
    # Ejemplo: si 'e' es índice 4 y vocab_size=21 → [0,0,0,0,1,0,0,...]
    xs[t] = np.zeros((vocab_size, 1))
    xs[t][inputs[t]] = 1
    
    # --- 1. Compuerta de actualización: ¿Cuánto actualizar? ---
    # Valores entre 0 (mantener todo el pasado) y 1 (usar toda la info nueva)
    zs[t] = sigmoid(np.dot(Wxz, xs[t]) + np.dot(Whz, hs[t-1]) + bz)
    
    # --- 2. Compuerta de reset: ¿Cuánto del pasado usar? ---
    # Valores entre 0 (olvidar pasado) y 1 (recordar todo el pasado)
    rs[t] = sigmoid(np.dot(Wxr, xs[t]) + np.dot(Whr, hs[t-1]) + br)
    
    # --- 3. Candidato de estado: ¿Qué información nueva proponer? ---
    # rs[t] * hs[t-1] filtra selectivamente el pasado antes de usarlo
    # tanh mantiene valores en rango [-1, 1] para estabilidad
    hs_tilde[t] = np.tanh(np.dot(Wxh, xs[t]) + 
                          np.dot(Whh, rs[t] * hs[t-1]) + bh)
    
    # --- 4. Nuevo estado: Mezcla inteligente de pasado y candidato ---
    # Si z=0.3 → 70% del estado anterior + 30% del candidato nuevo
    # Esto permite que la red aprenda a mantener información relevante por mucho tiempo
    hs[t] = (1 - zs[t]) * hs[t-1] + zs[t] * hs_tilde[t]
    
    # --- 5. Calcular predicción de siguiente carácter ---
    ys[t] = np.dot(Why, hs[t]) + by  # Puntuaciones sin normalizar (logits)
    
    # Convertir logits a probabilidades con softmax
    # Ejemplo: [2.1, 0.5, 3.2] → [0.28, 0.05, 0.67] (suman 1.0)
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
    
    # --- 6. Calcular pérdida (cross-entropy) ---
    # Si predijo correctamente (p≈1) → loss≈0 (bueno)
    # Si predijo mal (p≈0) → loss→∞ (malo)
    # El -log penaliza más las predicciones muy confiadas pero incorrectas
    loss += -np.log(ps[t][targets[t], 0])
  
  # ============================================================================
  # BACKWARD PASS: Calcular gradientes para ajustar los pesos
  # ============================================================================
  # Inicializar todos los gradientes en cero
  dWxz, dWhz, dbz = np.zeros_like(Wxz), np.zeros_like(Whz), np.zeros_like(bz)
  dWxr, dWhr, dbr = np.zeros_like(Wxr), np.zeros_like(Whr), np.zeros_like(br)
  dWxh, dWhh, dbh = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(bh)
  dWhy, dby = np.zeros_like(Why), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])  # Gradiente que fluye desde el futuro
  
  # Procesar en orden INVERSO porque los gradientes fluyen hacia atrás en el tiempo
  for t in reversed(range(len(inputs))):
    
    # --- Gradiente de la capa de salida ---
    # dy es la derivada de la función de pérdida respecto a las predicciones
    # Para softmax + cross-entropy: dy = p - y_verdadero
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1  # Restar 1 solo en la posición del carácter correcto
    
    # Acumular gradientes para Why y by
    dWhy += np.dot(dy, hs[t].T)  # ¿Cómo cambiar Why para reducir error?
    dby += dy                     # ¿Cómo cambiar by para reducir error?
    
    # --- Gradiente que llega al estado oculto ---
    # Tiene dos fuentes: 1) desde la salida actual, 2) desde el futuro (dhnext)
    dh = np.dot(Why.T, dy) + dhnext
    
    # --- Gradiente del candidato h_tilde ---
    # El candidato contribuyó al estado final proporcionalmente a z
    dh_tilde = dh * zs[t]
    # Derivada de tanh: d/dx[tanh(x)] = 1 - tanh²(x)
    dh_tilde_raw = (1 - hs_tilde[t] * hs_tilde[t]) * dh_tilde
    
    # Acumular gradientes del candidato
    dbh += dh_tilde_raw
    dWxh += np.dot(dh_tilde_raw, xs[t].T)
    dWhh += np.dot(dh_tilde_raw, (rs[t] * hs[t-1]).T)
    
    # --- Gradiente de la compuerta de reset ---
    # La reset gate afectó cuánto del pasado usar en el candidato
    dr = np.dot(Whh.T, dh_tilde_raw) * hs[t-1]
    # Derivada de sigmoid: d/dx[σ(x)] = σ(x) * (1 - σ(x))
    dr_raw = dr * rs[t] * (1 - rs[t])
    
    # Acumular gradientes de reset gate
    dbr += dr_raw
    dWxr += np.dot(dr_raw, xs[t].T)
    dWhr += np.dot(dr_raw, hs[t-1].T)
    
    # --- Gradiente de la compuerta de actualización ---
    # La update gate decidió cuánto usar del candidato vs. del pasado
    # Si el candidato era mejor que el pasado, z debería aumentar
    dz = dh * (hs_tilde[t] - hs[t-1])
    dz_raw = dz * zs[t] * (1 - zs[t])  # Derivada de sigmoid
    
    # Acumular gradientes de update gate
    dbz += dz_raw
    dWxz += np.dot(dz_raw, xs[t].T)
    dWhz += np.dot(dz_raw, hs[t-1].T)
    
    # --- Calcular gradiente para el paso temporal anterior ---
    # Este gradiente tiene 4 fuentes:
    # 1) Desde la mezcla directa (1-z)
    dhnext = dh * (1 - zs[t])
    # 2) Desde la update gate
    dhnext += np.dot(Whz.T, dz_raw)
    # 3) Desde la reset gate
    dhnext += np.dot(Whr.T, dr_raw)
    # 4) Desde el candidato (filtrado por reset)
    dhnext += np.dot(Whh.T, dh_tilde_raw) * rs[t]
  
  # --- Gradient clipping: Prevenir explosión de gradientes ---
  # Si un gradiente es muy grande (>5) o muy negativo (<-5), recortarlo
  # Esto evita actualizaciones descontroladas que arruinen el entrenamiento
  for dparam in [dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby]:
    np.clip(dparam, -5, 5, out=dparam)
  
  return loss, dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby, hs[len(inputs)-1]

In [11]:
def sample(h, seed_ix, n):
  """
  Genera texto usando el modelo entrenado.
  
  ¿CÓMO GENERA TEXTO LA GRU?
  1. Comienza con un carácter semilla (seed_ix)
  2. En cada paso:
     - Procesa el carácter actual con las compuertas GRU
     - Actualiza su estado interno (memoria)
     - Predice probabilidades para el siguiente carácter
     - Selecciona aleatoriamente según esas probabilidades
     - Usa ese carácter como entrada del siguiente paso
  3. Repite n veces
  
  La generación es ESTOCÁSTICA (no determinística) porque:
  - np.random.choice muestrea según probabilidades
  - Esto crea variedad: misma semilla → textos diferentes
  - Palabras/frases comunes se generan más frecuentemente
  
  Parámetros:
  - h: estado oculto inicial [hidden_size, 1]
  - seed_ix: índice del carácter para empezar
  - n: cantidad de caracteres a generar
  
  Retorna:
  - lista de índices de caracteres generados
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1  # Codificar carácter semilla en one-hot
  ixes = []
  
  for t in range(n):
    # Aplicar las tres operaciones de la GRU (igual que en entrenamiento)
    z = sigmoid(np.dot(Wxz, x) + np.dot(Whz, h) + bz)        # Update gate
    r = sigmoid(np.dot(Wxr, x) + np.dot(Whr, h) + br)        # Reset gate
    h_tilde = np.tanh(np.dot(Wxh, x) + np.dot(Whh, r * h) + bh)  # Candidato
    h = (1 - z) * h + z * h_tilde                            # Nuevo estado
    
    # Generar predicción
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))  # Probabilidades para cada carácter
    
    # Muestrear siguiente carácter según las probabilidades
    # Si p=['a':0.7, 'b':0.2, 'c':0.1] → 'a' se elige 70% del tiempo
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    
    # Preparar entrada para el siguiente paso
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  
  return ixes

In [12]:
n, p = 0, 0  # n = contador de iteraciones, p = posición actual en el texto

# Variables de memoria para ADAGRAD (optimizador adaptativo)
# Adagrad ajusta el learning rate individualmente para cada parámetro:
# - Parámetros con gradientes grandes → learning rate más pequeño
# - Parámetros con gradientes pequeños → learning rate más grande
# Esto ayuda a convergir más rápido y de forma más estable
mWxz, mWhz, mbz = np.zeros_like(Wxz), np.zeros_like(Whz), np.zeros_like(bz)
mWxr, mWhr, mbr = np.zeros_like(Wxr), np.zeros_like(Whr), np.zeros_like(br)
mWxh, mWhh, mbh = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(bh)
mWhy, mby = np.zeros_like(Why), np.zeros_like(by)

# Pérdida suavizada para visualización (evita fluctuaciones bruscas)
smooth_loss = -np.log(1.0/vocab_size)*seq_length  # Pérdida esperada si predijera al azar

print("Variables de entrenamiento inicializadas")
print(f"Pérdida inicial esperada: {smooth_loss:.2f}")

Variables de entrenamiento inicializadas
Pérdida inicial esperada: 74.89


In [13]:
max_iterations = 1000

while n < max_iterations:
  # --- Reiniciar si llegamos al final del texto o es la primera iteración ---
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1))  # Resetear memoria de la GRU
    p = 0  # Volver al inicio del texto
  
  # --- Preparar secuencias de entrada y objetivo ---
  # Entrada: caracteres en posiciones [p, p+1, ..., p+seq_length-1]
  # Objetivo: caracteres en posiciones [p+1, p+2, ..., p+seq_length]
  # Ejemplo: si seq_length=3 y texto="hola"
  #   inputs = ['h', 'o', 'l'] → targets = ['o', 'l', 'a']
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # --- Generar y mostrar texto de muestra cada 100 iteraciones ---
  # Esto nos permite ver cómo mejora el modelo durante el entrenamiento
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)  # Generar 200 caracteres
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # --- Calcular pérdida y gradientes ---
  loss, dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby, hprev = \
      lossFun(inputs, targets, hprev)
  
  # Suavizar pérdida: 99.9% del valor anterior + 0.1% del valor actual
  # Esto crea una curva suave para ver tendencias sin ruido
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: 
    print('iter %d, loss: %f' % (n, smooth_loss))
  
  # --- Actualizar parámetros usando Adagrad ---
  # Fórmula: θ = θ - (learning_rate / √(sum_gradientes² + ε)) * gradiente
  # El denominador aumenta con el tiempo → pasos más pequeños → convergencia estable
  params = [Wxz, Whz, bz, Wxr, Whr, br, Wxh, Whh, bh, Why, by]
  dparams = [dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby]
  mems = [mWxz, mWhz, mbz, mWxr, mWhr, mbr, mWxh, mWhh, mbh, mWhy, mby]
  
  for param, dparam, mem in zip(params, dparams, mems):
    mem += dparam * dparam  # Acumular cuadrado de gradientes
    # Actualizar parámetro con learning rate adaptativo
    # 1e-8 previene división por cero
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

  p += seq_length  # Avanzar al siguiente bloque de texto
  n += 1  # Incrementar contador de iteraciones

print("Entrenamiento GRU completado!")

----
 ofpoRxutipNRurcoaiRcbndtntacrNox cpvpxvrbNniocbTvtrboNiu NmnaaT t dTmremncbcTtnpraRTdnoinrt c oRxrroiRorcfiRb  bdcvuuervx fdrivirpmoevin iomNurveoxi rxmnRiin eoc acvptu fmna axitepeRinene RbncmartoRpN 
----
iter 0, loss: 74.893182
----
 RcNvNTrcoiuxmap pdpefiofrrveiR mibNcrdvvNuupvnNa ornrvdeaxedTepm aRtmntpmafeermxrdtf RurRTeu rveamcri RuxpmrcoT emvRpudtccNxbibudmodio dTtramedm pxfNrerdn pNnaRvccprnfTippnraxdpnnNinbcRtoRv bxRviuaoct 
----
iter 100, loss: 74.875727
----
 nnvbodmRneNpedcutvNermiRvpdfRdd u xvtxoadxebmRRpTpRpTmc btv mdtTfvnRmupvdbdTnoenaxupubtNRRrfoiiRxamxecrNRbeebRpmrTtvpbnntpxrmaTxofeeiNeiacuubpdeNtfuRriNiNtdcdafpr NbvoxrfaprvcmopRpacuttbtaabtbTc rxaTo 
----
iter 200, loss: 74.854022
----
 arfboRbiRvepdaanppToRffdoeuafRmrtmudefxxupfapbuToNur nnboovNxRn nNvafNiubNRbreircfoTmbmuprupvvntvpRetNNtRmcvtNppdpiNatmivnntcnpnpeaxcavTcux orcpeNfR rvdctRRixroxtcavtrriduToNRnoaaxuepvnutpvr RrpentcbT 
----
iter 300, loss: 74.829354
----
 puiuamidddepcvrNmRtToNitnm mRvvbRuoec o 